In [20]:
import meld_graph
import meld_graph.models
import meld_graph.experiment
import meld_graph.dataset
import meld_graph.data_preprocessing
import meld_graph.evaluation


import importlib
importlib.reload(meld_graph)
importlib.reload(meld_graph.models)
importlib.reload(meld_graph.dataset)
importlib.reload(meld_graph.experiment)
importlib.reload(meld_graph.data_preprocessing)
importlib.reload(meld_graph.evaluation)

import logging
import os
import json

from meld_graph.dataset import GraphDataset, Oversampler
from meld_classifier.meld_cohort import MeldCohort, MeldSubject
from meld_graph.training import Metrics
import numpy as np
from meld_graph.paths import EXPERIMENT_PATH

from meld_graph.evaluation import Evaluator



### testing the class

In [21]:
#load experiment already trained using checkpoint path

EXPERIMENT_PATH='/rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1'
checkpoint_path=os.path.join(EXPERIMENT_PATH, '22-08-22_synth_unet_gamma_2_synthetic_data__bias/synthetic_data__bias_1/fold_00')

exp = meld_graph.experiment.Experiment.from_folder(checkpoint_path)


Initialised Experiment 22-08-22_synth_unet_gamma_2_synthetic_data__bias/synthetic_data__bias_1


In [22]:
# update experiment with the test data parameters you want to run on

exp.data_parameters['synthetic_data']['n_subs'] = 8

In [23]:
# Run the evaluation on the test data and save into directory provided

save_dir=os.path.join(EXPERIMENT_PATH,'22-08-22_synth_eval_bis')

eva = Evaluator(experiment = exp,
                checkpoint_path = checkpoint_path,
                save_dir = save_dir ,
                make_images = True,
               )


Using coord type exact
Loading model weights from checkpoint /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1/22-08-22_synth_unet_gamma_2_synthetic_data__bias/synthetic_data__bias_1/fold_00/best_model.pt


conv 33 16
conv 16 16
conv 16 16


In [24]:
eva.evaluate()

loading data and predicting model
Loading and preprocessing data
Combine hemis None
Using coord type pseudo


## Evaluation using baseline model

In [ ]:
# 
import torch
exp.load_model(checkpoint_path='/rds/project/kw350/rds-kw350-meld/experiments_graph/co-spit1/22-07-29_synth_3layer/finetune_radius0.2/fold_00/best_model.pt', 
               force=True )

In [ ]:
data_loader = torch_geometric.loader.DataLoader(dataset,
            shuffle=False, )

In [ ]:
#predict 
import torch_geometric.data
from meld_graph.training import Metrics

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
data_loader = torch_geometric.loader.DataLoader(dataset,
            shuffle=False, batch_size=2)
metrics = Metrics(params['metrics'])  # for keeping track of running metrics
for data in data_loader:
    print('loader')
    data = data.to(device)
    estimates = exp.model(data.x)
    labels = data.y.squeeze()   
    # metrics
    pred = torch.argmax(estimates[0], axis=1)
    # update running metrics
    metrics.update(pred, labels)

metrics.get_aggregated_metrics()

In [ ]:
#plot lesion
import matplotlib_surface_plotting as msp
from meld_graph.icospheres import IcoSpheres

#initialise the icosphere
icos=IcoSpheres(conv_type='SpiralConv')
ico_index=7
ico_ini = icos.icospheres[ico_index]

#plot example of feature on icosphere for 1 subject
msp.plot_surf(ico_ini['coords'],
              ico_ini['faces'],
              pred.numpy()[0:163842], 
              parcel=labels.numpy()[0:163842], 
              parcel_cmap={1:[1,0,0,0]},
              colorbar=False, rotate=[50,250], 
                 )


In [ ]:
#plot lesion
import matplotlib_surface_plotting as msp
from meld_graph.icospheres import IcoSpheres

#initialise the icosphere
icos=IcoSpheres(conv_type='SpiralConv')
ico_index=7
ico_ini = icos.icospheres[ico_index]

#plot example of feature on icosphere for 1 subject
index = 3
feat=1
features_subj, labels_subj = dataset.data_list[index]
msp.plot_surf(ico_ini['coords'],
              ico_ini['faces'],
              features_subj[:,feat], 
              parcel=labels_subj, 
              parcel_cmap={1:[1,0,0,0]},
              colorbar=False, rotate=[150,250], 
                 )
